In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
from local_functions import *
import math
import re
import time

In [ ]:
main_dir = Path(os.getcwd())
data_path = main_dir / "data"
processed_data_path = data_path / 'processed'

In [ ]:
fid_cols = ['gameId', 'playId', 'frameId']
playerframe_cols = fid_cols + ['nflId']

In [ ]:
df_tracking = pd.read_parquet(processed_data_path / 'tracking.parquet').reset_index(drop=True)
df_model_results = pd.read_parquet(processed_data_path / 'model_results.parquet')

In [ ]:
#create unique frame Id
df_tracking['game_play_frame_id'] = df_tracking['gameId'].apply(str) + '_' + df_tracking['playId'].apply(str) + '_' + df_tracking['frameId'].apply(str)
df_model_results['game_play_frame_id'] = df_model_results['gameId'].apply(str) + '_' + df_model_results['playId'].apply(str) + '_' + df_model_results['frameId'].apply(str)

In [ ]:
# Filter tracking for frames that were modeled
dft_modeljoin = df_tracking[df_tracking['game_play_frame_id'].isin(df_model_results['game_play_frame_id'])]

# Filter for offensive players and football which weren't in model
dft_modeljoin = dft_modeljoin[(dft_modeljoin['on_offense']) | (dft_modeljoin['nflId'].isnull())]

In [ ]:
# Join tracking data points to model results required for play animation
df_model_results_track = df_model_results.merge(df_tracking.loc[:, playerframe_cols+['x_std', 'y_std', 'o_std', 'dir_std', 'club', 'event', 'jerseyNumber', 'is_ballcarrier']], on = playerframe_cols)
df_model_results_track = df_model_results_track.loc[:, playerframe_cols+['influence', 'in_block', 'xtackle', 'xtackle_xgb', 'x_std', 'y_std', 's', 'o_std', 'dir_std', 'club', 'event', 'jerseyNumber', 'is_ballcarrier']]

In [ ]:
# In filtered tracking data, select only required/matching columns to concat onto model results for play animation
dft_modeljoin = dft_modeljoin.loc[:, playerframe_cols+['x_std', 'y_std', 'o_std', 'dir_std', 's', 'club', 'event', 'jerseyNumber', 'is_ballcarrier']].reset_index(drop=True)

In [ ]:
df_tracking_with_results = pd.concat([dft_modeljoin, df_model_results_track]).reset_index(drop=True)
df_tracking_with_results['game_play_id'] = df_tracking_with_results['gameId'].apply(str) + '_' + df_tracking_with_results['playId'].apply(str)

In [ ]:
#example_play = df_tracking_with_results.sort_values('xtackle_xgb', ascending=False).loc[:,'game_play_id'].iloc[1]
example_play = df_tracking_with_results.game_play_id.sample(1).iloc[0]

In [ ]:
df_tracking_with_results['side_of_ball'] = np.where(df_tracking_with_results['nflId'].isnull(), 
                                                    'football', 
                                                    np.where(df_tracking_with_results['xtackle_xgb'].isnull(), 'offense', 'defense'))

In [ ]:
df_tracking_with_results['xtackle_xgb'] = df_tracking_with_results['xtackle_xgb'].fillna(0)

In [ ]:
example_play = '2022091101_2501' # toss left

In [ ]:
animate_tracking_data(tracking_df = df_tracking_with_results, 
                      id_game_play = example_play,
                      x_col = 'x_std',
                      y_col = 'y_std',
                      dir_col = 'dir_std',
                      dir_arrow_metric = 's',
                      o_col = 'o_std')

In [ ]:
example_frame = df_tracking_with_results[(df_tracking_with_results['game_play_id'] == example_play) & (df_tracking_with_results['frameId'] == 1)]

In [ ]:
example_frame

In [ ]:
def animate_tracking_data_probabilities(tracking_df, id_game_play, x_col, y_col, dir_col, dir_arrow_metric, o_col):

    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from matplotlib.animation import FuncAnimation
    from IPython.display import display, HTML
    import matplotlib.patches as patches
    import math

    n = tracking_df[tracking_df.game_play_id == id_game_play].frameId.max()

    # Initialize the football field plot
    fig, ax = create_football_field(return_fig=True)

    max_x = tracking_df[tracking_df.game_play_id == id_game_play].x_std.max()
    min_x = tracking_df[tracking_df.game_play_id == id_game_play].x_std.min()

    ax.set_xlim(min_x-5,max_x+5)

    # # Get unique club names and assign colors
    # clubs = tracking_df[(tracking_df.game_play_id == id_game_play) & (tracking_df['club'] != 'football')]['club'].unique()
    # print(clubs)
    # club_colors = {clubs[0]: 'orange', clubs[1]: 'lightblue', 'football': 'brown'}

    dots = []
    texts = []  # To store jersey number text elements
    dir_arrows = []
    o_arrows = []

    # # Initialize the scatter plot for each club.
    # scatters = {}
    # for sob in tracking_df.side_of_ball.unique():
    #     if sob == "football":
    #         scatters[sob] = ax.scatter([], [], label=sob, s=80, color='brown', lw=1, edgecolors="black", zorder=5)
    #     elif sob == 'offense':
    #         scatters[sob] = ax.scatter([], [], label=sob, s=170, color='grey', lw=1, edgecolors="black", zorder=5)
    #     else:
    #         scatters[sob] = ax.scatter([], [], label=sob, s=170, color='white', lw=1, edgecolors="black", zorder=5)
            
    #ax.legend().remove()

    def update(frame):
        # Clear previous frame's texts

        for dot in dots:
            dot.remove()
        dots.clear()

        for text in texts:
            text.remove()
        texts.clear()

        for arrow in dir_arrows:
            arrow.remove()
        dir_arrows.clear()

        for arrow in o_arrows:
            arrow.remove()
        o_arrows.clear()
        

        frame_data = tracking_df[(tracking_df.game_play_id == id_game_play) & (tracking_df.frameId == frame)]
        # event_for_frame = frame_data['event'].iloc[0]  # Assuming each frame has consistent event data

        # if pd.notna(event_for_frame):
        #     ax.set_title(f"Tracking data for {id_game_play}: at frame {frame}\nEvent: {event_for_frame}", fontsize=15)
        # else:
        #     ax.set_title(f"Tracking data for {id_game_play}: at frame {frame}", fontsize=15)

        for sob, d in frame_data.groupby('side_of_ball'):
            # scatters[sob].set_offsets(np.c_[d[x_col].values, d[y_col].values])
            # if sob == 'football':
            #     scatters[sob].set_color('brown')
            # elif sob == 'offense':
            #     scatters[sob].set_color('grey')
            # else:
            #     scatters[sob].set_color(d['xtackle_xgb'])

            # scatters[sob].set_edgecolors("black")  # Explicitly setting the edge color
            
            if sob=='football':
                dot = ax.scatter(d[x_col], d[y_col], c = 'brown', s = 80, edgecolors="black")
                dots.append(dot)
                
            elif sob=='offense':
                dot = ax.scatter(d[x_col], d[y_col], c = ['blue' if value == 1 else 'lightblue' for value in d['is_ballcarrier']], 
                                 s = 170, edgecolors="black")
                dots.append(dot)
            else:
                dot = ax.scatter(d[x_col], d[y_col], c = d['xtackle_xgb'], cmap='Oranges', s = 170, edgecolors="black")
                dot.set_clim([0,1])
                dots.append(dot)


            # Display jersey numbers if it's not the football
            if sob != "football":                    
                
                for _, row in d.iterrows():
                    try:

                        text = ax.text(row[x_col], row[y_col], str(int(row["jerseyNumber"])), 
                                   fontsize=8, ha='center', va='center', color="black", fontweight='bold', zorder=6)
                        texts.append(text)

                        #direction arrows
                        dir_angle = math.radians(90-row[dir_col])

                        dir_dx = row[dir_arrow_metric] * 0.75 * math.cos(dir_angle)
                        dir_dy = row[dir_arrow_metric] * 0.75 * math.sin(dir_angle)

                        dir_arrow = ax.quiver(row[x_col], row[y_col], dir_dx, dir_dy, angles='xy', scale_units='xy', width = 0.004, scale=1, alpha=0.5, color = 'orange')
                        dir_arrows.append(dir_arrow)

                        #orientation arrows
                        o_angle = math.radians(90-row[o_col])

                        o_dx = 0.75 * math.cos(o_angle)
                        o_dy = 0.75 * math.sin(o_angle)

                        o_arrow = ax.quiver(row[x_col], row[y_col], o_dx, o_dy, angles='xy', scale_units='xy', width = 0.0088, scale=1, alpha=0.5, color = 'blue')
                        o_arrows.append(o_arrow)
                        
                    except ValueError:
                        continue

    ani = FuncAnimation(fig, update, frames=range(1, n+1), repeat=True, interval=200)
    ani.save('figs/animation_'+example_play+'.gif', writer='pillow', fps=10)

    plt.close(ani._fig)

    # Display the animation in the notebook
    return HTML(ani.to_jshtml())

In [ ]:
animate_tracking_data_probabilities(tracking_df = df_tracking_with_results, 
                      id_game_play = example_play,
                      x_col = 'x_std',
                      y_col = 'y_std',
                      dir_col = 'dir_std',
                      dir_arrow_metric = 's',
                      o_col = 'o_std')

In [ ]:
df_block_wresults = df_model_results[df_model_results['in_block']==1]#.groupby(playerframe_cols)

In [ ]:
df_block_wresults = df_block_wresults.sort_values(['gameId', 'playId', 'nflId', 'frameId']).assign(last_frame=df_block_wresults.groupby(['gameId', 'playId', 'nflId'])['frameId'].shift(1))

In [ ]:
df_block_wresults['new_block'] = np.where((df_block_wresults['last_frame'].notnull()) & (df_block_wresults['last_frame'] != df_block_wresults['frameId']-1), 1, 0)


In [ ]:
df_block_wresults['new_block_counter'] = df_block_wresults.groupby(['gameId', 'playId', 'nflId'])['new_block'].cumsum()

In [ ]:
def val_at_min_max(group, id_col, val_col):
    min_id = group[id_col].idxmin()
    max_id = group[id_col].idxmax()
    return pd.Series({'val_at_min_id': group.loc[min_id, val_col], 'val_at_max_id': group.loc[max_id, val_col]})

In [ ]:
df_block_shed = df_block_wresults.groupby(['gameId', 'playId', 'nflId', 'new_block_counter']).apply(val_at_min_max, id_col='frameId', val_col='xtackle_xgb').reset_index()

In [ ]:
df_block_shed

In [ ]:
df_block_shed['prob_change'] = df_block_shed['val_at_max_id'] - df_block_shed['val_at_min_id']

In [ ]:
df_block_shed_agg = df_block_shed.groupby('nflId').agg({'playId' : 'count', 'prob_change' : 'sum'}).reset_index().rename(columns = {'playId' : 'blocks'})

In [ ]:
df_block_shed_agg['avg_prob_change'] = df_block_shed_agg['prob_change'] / df_block_shed_agg['blocks']

In [ ]:
df_players = pd.read_csv(f'{data_path}/players.csv')

In [ ]:
df_players

In [ ]:
df_block_shed_table = df_block_shed_agg.merge(df_players.loc[:,['nflId', 'displayName']], on = 'nflId')

In [ ]:
df_block_shed_table.blocks.hist()

In [ ]:
df_block_shed_table = df_block_shed_table.sort_values('avg_prob_change', ascending=False).reset_index(drop=True)

In [ ]:
df_block_shed_table.to_csv(processed_data_path / 'block_shed_agg.csv')